In [6]:
import math

Q96 = 2**96
MAX_DELTA = 0.001


def lt_delta(x, y, d):
    return abs(x - y) < d


# Tick to sqrt price x 96
def tick_to_s96(tick):
    return int(1.0001 ** (tick / 2) * Q96)


def calc_x(L, s):
    assert s >= 0
    if s == 0:
        return math.inf
    return L / s


def calc_y(L, s):
    assert s >= 0
    return L * s


def calc_dx(L, s_lo, s_hi):
    assert s_lo >= 0
    assert s_hi >= 0
    if s_lo >= s_hi:
        return 0
    if s_lo == 0:
        return math.inf
    # 1 / math.inf = 0
    return L * (1 / s_lo - 1 / s_hi)


def calc_dy(L, s_lo, s_hi):
    assert s_lo >= 0
    assert s_hi >= 0
    if s_lo >= s_hi:
        return 0
    return L * (s_hi - s_lo)


# dx = L(1/s_lo - 1/s_hi)
# dy = L(s_hi - s_lo)
def calc_dx_to_s_lo(L, s_hi, dx):
    assert s_hi > 0
    assert dx >= 0
    return 1 / (dx / L + 1 / s_hi)


def calc_dx_to_s_hi(L, s_lo, dx):
    assert s_lo > 0
    assert dx >= 0
    return 1 / (-dx / L + 1 / s_lo)


def calc_dy_to_s_lo(L, s_hi, dy):
    assert s_hi > 0
    assert dy >= 0
    return -dy / L + s_hi


def calc_dy_to_s_hi(L, s_lo, dy):
    assert s_lo > 0
    assert dy >= 0
    return dy / L + s_lo


# next
def nxt(pool, i, up):
    if i + 1 >= len(pool):
        if up:
            return (math.inf, math.inf, 0)
        else:
            return (0, 0, 0)
    return pool[i + 1]


def calc_amt_out(i, o, di, f):
    di *= 1 - f
    return o * di / (i + di)


# Optimal dy amount in (including fees) into pool A (dya -> dx -> dyb)
def calc_opt_dy_in(xa, ya, xb, yb, fa, fb):
    k0 = xa * ya * xb * yb * (1 - fa) * (1 - fb)
    k1 = (xb + xa * (1 - fb)) * (1 - fa)
    a = k1 * k1
    b = 2 * k1 * xb * ya
    c = (xb * ya) ** 2 - k0
    return (-b + math.sqrt(b * b - 4 * a * c)) / (2 * a)


def calc_opt_dya(la, sa, lb, sb, fa, fb):
    assert 0 <= sa <= sb
    xa = calc_x(la, sa)
    ya = calc_y(la, sa)
    xb = calc_x(lb, sb)
    yb = calc_y(lb, sb)
    dya = calc_opt_dy_in(xa, ya, xb, yb, fa, fb)
    assert dya >= 0
    dx = calc_amt_out(ya, xa, dya, fa)
    dyb = calc_amt_out(xb, yb, dx, fb)
    assert dya <= dyb
    # Check sa and sb after swaps
    sa_swap = calc_dy_to_s_hi(la, sa, dya * (1 - fa))
    sb_swap = calc_dx_to_s_lo(lb, sb, dx * (1 - fb))
    assert sa <= sa_swap <= sb_swap <= sb
    assert lt_delta(sa_swap, sb_swap, MAX_DELTA)
    return (dya, dyb, sa_swap)


def swap_to_sa_hi(xa, xb, la, sa, sa_lo, sa_hi, lb, sb, sb_lo, sb_hi, fa, fb):
    dya = calc_dy(la, sa_lo, sa_hi) / (1 - fa)
    sa = sa_hi
    sb = calc_dx_to_s_lo(lb, sb_hi, xa * (1 - fb))
    assert sb_lo <= sb
    dyb = calc_dy(lb, sb, sb_hi)
    return (dya, dyb, sa, sb)


def swap_to_sb_lo(xa, xb, la, sa, sa_lo, sa_hi, lb, sb, sb_lo, sb_hi, fa, fb):
    dx = min(xa, xb / (1 - fb))
    if dx == xa:
        sa = sa_hi
    else:
        sa = calc_dx_to_s_hi(la, sa_lo, dx)
        assert sa <= sa_hi
    dya = calc_dy(la, sa_lo, sa) / (1 - fa)
    if dx == xa:
        sb = calc_dx_to_s_lo(lb, sb_hi, xa * (1 - fb))
        assert sb_lo <= sb
    else:
        sb = sb_lo
    dyb = calc_dy(lb, sb, sb_hi)
    return (dya, dyb, sa, sb)


# pa < pb
# dya -> dx -> dyb
# TODO: dxb -> dy -> dxa
# pool = [(sqrt price lo, sqrt price hi, liquidity)]
def calc_dya(pool_a, pool_b, fa, fb):
    (sa_lo, sa_hi, la) = pool_a[0]
    (sb_lo, sb_hi, lb) = pool_b[0]
    a = 0
    b = 0
    dya = 0
    dyb = 0
    sa = sa_lo
    sb = sb_hi

    while sa_lo < sb_hi:
        xa = calc_dx(la, sa_lo, sa_hi)
        xb = calc_dx(lb, sb_lo, sb_hi)
        if sa_hi <= sb_lo:
            if xa <= xb:
                # swap to sa_hi
                (da, db, sa, sb) = swap_to_sa_hi(
                    xa, xb, la, sa, sa_lo, sa_hi, lb, sb, sb_lo, sb_hi, fa, fb
                )
                dya += da
                dyb += db
            else:
                # swap to sb_lo
                (da, db, sa, sb) = swap_to_sb_lo(
                    xa, xb, la, sa, sa_lo, sa_hi, lb, sb, sb_lo, sb_hi, fa, fb
                )
                dya += da
                dyb += db
        else:
            (dya_opt, dyb_opt, s_opt) = calc_opt_dya(la, sa, lb, sb, fa, fb)
            if sa_hi < s_opt:
                # swap to sa_hi
                (da, db, sa, sb) = swap_to_sa_hi(
                    xa, xb, la, sa, sa_lo, sa_hi, lb, sb, sb_lo, sb_hi, fa, fb
                )
                dya += da
                dyb += db
            elif s_opt < sb_lo:
                # swap to sb_lo
                (da, db, sa, sb) = swap_to_sb_lo(
                    xa, xb, la, sa, sa_lo, sa_hi, lb, sb, sb_lo, sb_hi, fa, fb
                )
                dya += da
                dyb += db
            else:
                # swap to s_opt
                dya += dya_opt
                dyb += dyb_opt
                sa_lo = s_opt
                sb_hi = s_opt
                sa = s_opt
                sb = s_opt
        if sb <= sa:
            break
        if sa == sa_hi:
            (sa_lo, sa_hi, la) = nxt(pool_a, a, True)
            a += 1
        else:
            assert sa_lo <= sa
            sa_lo = sa
        if sb == sb_lo:
            (sb_lo, sb_hi, lb) = nxt(pool_b, b, False)
            b += 1
        else:
            assert sb <= sb_hi
            sb_hi = sb

    assert dya <= dyb
    assert sa <= sb
    return (dya, dyb, sa, sb)

In [7]:
pool_a = [(math.sqrt(100), math.inf, 1000)]
pool_b = [(0, math.sqrt(200), 2000)]
fa = 0
fb = 0
calc_dya(pool_a, pool_b, fa, fb)

(2426.4068711928517,
 3431.4575050761987,
 12.426406871192851,
 12.426406871192851)

In [ ]:
import random
from collections import defaultdict

import matplotlib.pyplot as plt
import numpy as np

# | Fee Tier | Typical Tick Spacing |
# | -------- | -------------------- |
# | 0.05%    | 1                    |
# | 0.3%     | 60                   |
# | 1%       | 200                  |
# | 10%      | 2000                 |

TICK_SPACING_A = 60
TICK_SPACING_B = 60

TICK_A = -100
TICK_B = 100


def random_positions(
    n_positions,
    tick_min=-500,
    tick_max=500,
    tick_spacing=10,
    min_width=20,
    max_width=200,
    min_liquidity=500,
    max_liquidity=6000,
):
    positions = []

    # snap tick_min and tick_max to tick spacing
    tick_min = (tick_min // tick_spacing) * tick_spacing
    tick_max = (tick_max // tick_spacing) * tick_spacing

    for _ in range(n_positions):
        # generate lower tick as multiple of tick_spacing
        lower = (
            random.randint(
                tick_min // tick_spacing, (tick_max - min_width) // tick_spacing
            )
            * tick_spacing
        )

        # random width in ticks, then snap upper
        width = random.randint(min_width, max_width)
        upper = lower + width
        # snap upper to tick spacing
        upper = ((upper + tick_spacing - 1) // tick_spacing) * tick_spacing
        if upper > tick_max:
            upper = tick_max

        liquidity = random.randint(min_liquidity, max_liquidity)
        positions.append((lower, upper, liquidity))

    return positions


def build_liq(positions):
    liquidity_net = defaultdict(float)

    for lower, upper, L in positions:
        liquidity_net[lower] += L
        liquidity_net[upper] -= L

    ticks = np.array(sorted(liquidity_net.keys()))
    delta = np.array([liquidity_net[t] for t in ticks])
    active = np.cumsum(delta)

    return ticks, active


pool_A_positions = random_positions(n_positions=4, tick_spacing=TICK_SPACING_A)
pool_B_positions = random_positions(n_positions=4, tick_spacing=TICK_SPACING_B)

ticks_A, liq_A = build_liq(pool_A_positions)
ticks_B, liq_B = build_liq(pool_B_positions)

print(ticks_A, liq_A)

plt.figure(figsize=(10, 4))

plt.step(ticks_A, liq_A, where="post", linewidth=2, label="Pool A")

plt.step(ticks_B, liq_B, where="post", linewidth=2, label="Pool B")

plt.axvline(TICK_A, linewidth=1.5, color="tab:pink", label=f"TICK_A = {TICK_A}")
plt.axvline(TICK_B, linewidth=1.5, color="tab:pink", label=f"TICK_B = {TICK_B}")

plt.xlabel("Tick")
plt.ylabel("Liquidity")
plt.title("Uniswap v3 pools")

plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()